In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

stop_Words = text.ENGLISH_STOP_WORDS


In [24]:
df = pd.read_csv("./data/parsed/training.csv", dtype=str)
df.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [3]:
import nltk, re

In [36]:
words = set(nltk.corpus.words.words())

#df["Message-Contents"].apply(len)
#df["Message-Contents"][16]
gross_email = df[df["Message-Contents"].str.contains("----")]


In [11]:
import re

'''
Email filter class that will hold all of our regexes that we can apply to ALL emails. It is helpful 
to have it as an objectbecause we can just add the regexes as instance variables and then iterate 
over all of them using <object>.__dict__.items()
'''
class Email_filter:
     def __init__(self):
        ##regexes#
        self.quoted = re.compile(r"^>(\s)*", re.MULTILINE)
        self.original_or_forward = re.compile(r"(^(\s)*-*(\s)*Original Message(\s)*-.*$)|(^____.*$)", re.MULTILINE)
        self.metadata = re.compile(r"(^From:.*$)|(^Sent:.*$)|(^To:.*$)|(^Subject:.*$)|(^Cc:.*$)|(^Date:.*$)|(^Encoding:.*$)", re.MULTILINE)
        self.ole_object = re.compile(r"(<<.*?>>)+", re.DOTALL|re.MULTILINE)
        self.smtp_header = re.compile(r"^Message-id:.*?X-Mozilla-Status.*?$", re.DOTALL|re.MULTILINE)
        self.received = re.compile(r"^Received:(.*?)\([A-Z]{3}\)", re.DOTALL|re.MULTILINE)

'''
Method to actually filter the emails - creates an email filter object and loops through all of the 
regexes it contains, running each of them on an email. If the email is a reply, it will first 
cut out all of the replies.
'''
def filter_email(s):
    if type(s) != str:
        return("")
    e = Email_filter()
    ret = s
    for variable_name, regex in e.__dict__.items():  
        ret = re.sub(regex, "", ret)
    
    return ret

'''
Method to cut out all of the replies of emails
'''
def filter_reply(s):
    replies = re.compile(r"(^(\s)*-*(\s)*Original Message.*)", re.MULTILINE|re.DOTALL)
    ret = re.sub(replies, "", s)
    return ret


'''
metadata parsing
'''
from datetime import datetime
import re
def parse_date(d):
    d = d.replace(',', '')
    redundancy_filter = d.split(' (')
    string = redundancy_filter[0]
    datetime_object = datetime.strptime(string, '%a %d %b %Y %X %z')
    return datetime_object


def x_strip(s):
    if s != "":
        array = re.split(r'</O.*?\>', s)
        array = [x.strip() for x in array]
        return array
    else:
        return([])

def to_from_strip(s):
    if s != "":
        return s.split(',')
    else:
        return([])
    
def parse_metadata(df):
    df['Date'] = df['Date'].apply(parse_date)
    df['To'] = df['To'].apply(to_from_strip)
    df['From'] = df['From'].apply(to_from_strip)
    df['X-To'] = df['X-To'].apply(x_strip)
    df['X-From'] = df['X-From'].apply(x_strip)
    
    return(df)
    
def full_filter_email(df):
    df = df.replace(np.nan, '', regex=True)
    df.loc[df["Subject"].str.lower().str.startswith('re')]["Message-Contents"].apply(filter_reply)
    df["Message-Contents"] = df["Message-Contents"].apply(filter_email)
    df = parse_metadata(df)
    return(df)

In [5]:
gross_email_filtered = pd.DataFrame()
gross_email_filtered = full_filter_email(df)

NameError: name 'full_filter_email' is not defined

In [17]:
print(gross_email_filtered["Message-Contents"][102])

revised.







                                          AGENT TR/TR BOOKING REF ZNUHAJ
 
                                          ENRON NORTH AMERICA
                                          CAUSHOLLI/MONIKA
                                          3WTC0306
  ENRON CORP                              INTEROFFICE ETKT RCPT
 
 
 
 
 
  DATE:  OCT 03 2001
 
 
 
HOTEL                 21OCT CORONADO BAY RESORT
                      23OCT 4000 CORONADO BAY ROAD
                            CORONADO, CA 92118
                            UNITED STATES OF AMERICA
                            TELEPHONE: 619 424 4000
                            FAX: 619 424 4400
                            CONFIRMATION: 95683347
                            REFERENCE: A0DAD2
                            SINGLE ROOM KING SIZE BED
                            RATE: *** USD 160.00 PER NIGHT
                            GUARANTEE GIVEN
                            ENRON CORP RQST NONSMKG KING
         TO AVOID BILLING CA

In [6]:
gross_email_filtered.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [7]:
gross_email_filtered.to_pickle('data/parsed/pickles/pickled_data_test.pickle')

In [13]:
print(gross_email["Subject"][50846])
print(filter_email(gross_email["Message-Contents"][50846], is_reply=False))

RE: Transfer of Dana Davis


TypeError: filter_email() got an unexpected keyword argument 'is_reply'

In [29]:
print(gross_email["Message-Contents"][50846])
print(gross_email_filtered["Message-Contents"][50846])

NameError: name 'gross_email' is not defined

## TF-IDF

In [4]:

# emails = pd.read_pickle("data/parsed/pickles/pickled_data_test.pickle")

#scenario_1 = df[gross_email_filtered["Scenario"] == '401']
scenario_1 = pd.DataFrame(gross_email_filtered[0::3])
scenario_2 = pd.DataFrame(gross_email_filtered[1::3])
scenario_3 = pd.DataFrame(gross_email_filtered[2::3])

def build_TFIDF_Matrix(df):
    vectorizer = TfidfVectorizer(stop_words = stop_Words, min_df = .0005)
    vectorized = vectorizer.fit_transform(df["Message-Contents"])

    dumb_numbers = [s for s in vectorizer.get_feature_names()
    if (("0" in s) or ("1" in s) or ("2" in s) or ("3" in s) or ("4" in s)
    or ("5" in s) or ("6" in s) or ("7" in s) or ("8" in s) or ("9" in s) or ("_" in s) or s == "ll")]

    stop_words = text.ENGLISH_STOP_WORDS.union(dumb_numbers)

    vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = .0005)
    vectorized = vectorizer.fit_transform(df["Message-Contents"])
    return vectorizer, vectorized

NameError: name 'gross_email_filtered' is not defined

In [27]:
print(scenario_1["Message-Contents"][102])

revised.







                                          AGENT TR/TR BOOKING REF ZNUHAJ
 
                                          ENRON NORTH AMERICA
                                          CAUSHOLLI/MONIKA
                                          3WTC0306
  ENRON CORP                              INTEROFFICE ETKT RCPT
 
 
 
 
 
  DATE:  OCT 03 2001
 
 
 
HOTEL                 21OCT CORONADO BAY RESORT
                      23OCT 4000 CORONADO BAY ROAD
                            CORONADO, CA 92118
                            UNITED STATES OF AMERICA
                            TELEPHONE: 619 424 4000
                            FAX: 619 424 4400
                            CONFIRMATION: 95683347
                            REFERENCE: A0DAD2
                            SINGLE ROOM KING SIZE BED
                            RATE: *** USD 160.00 PER NIGHT
                            GUARANTEE GIVEN
                            ENRON CORP RQST NONSMKG KING
         TO AVOID BILLING CA

In [29]:
scenario_1_tfidf_matrix = build_TFIDF_Matrix(scenario_1)
feature_names = scenario_1_tfidf_matrix[0].get_feature_names()
scenario_1_tfidf = scenario_1_tfidf_matrix[1].toarray()
# print(scenario_1_tfidf_matrix[1].toarray().shape)
# 
# np.save('scenario_1_tfidf.npy', scenario_1_tfidf)
# print(type(feature_names))
#scenario_1_data_frame = pd.DataFrame(data = scenario_1_tfidf, index = scenario_1["ID"], columns = scenario_1_tfidf_matrix[0].get_feature_names())
# scenario_1_data_frame.to_pickle('data/parsed/pickles/pickled_scenario_1_tfidf.pickle')



In [31]:
np.save('scenario_1_tfidf.npy', scenario_1_tfidf)

In [30]:
print(len(feature_names))
print(feature_names)

11587
['aa', 'aaa', 'aaron', 'ab', 'abag', 'abandon', 'abandoned', 'abandonment', 'abb', 'abbreviation', 'abc', 'abel', 'abide', 'abilities', 'ability', 'able', 'abn', 'aboard', 'abraham', 'abramo', 'abrams', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'abuse', 'ac', 'academic', 'academy', 'accelerate', 'accelerated', 'acceleration', 'accenture', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessible', 'accessing', 'accessories', 'accident', 'accidentally', 'accommodate', 'accommodations', 'accomodate', 'accompanied', 'accompany', 'accompanying', 'accomplish', 'accomplished', 'accomplishments', 'accord', 'accordance', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accountants', 'accounted', 'accounting', 'accounts', 'accrual', 'accrue', 'accrued', 'acct', 'accumulated', 'accuracy', 'accurate', 'accurately', 'accused', 'ace', 'achieve', 'achieved', 'achieving', 'ackerman',

In [10]:
# for i in range(0,33,3):
#     print("I =", i)
#     print(scenario_1["Message-Contents"][i])
#     for j in range(len(feature_names)):
#         iJ = i // 3
#         if scenario_1_tfidf[iJ][j] != 0:
#             print(feature_names[j])
#             print(scenario_1_tfidf[iJ][j])
print(scenario_1["Message-Contents"][102])
for i in range(len(feature_names)):
    if scenario_1_tfidf[102][i] != 0:
        print(feature_names[i])
        print(scenario_1_tfidf[102][i])

revised.

-----Original Message-----
From: Teresa Robertson [mailto:teresa.robertson@travelpark.com]
Sent: Wednesday, October 03, 2001 2:03 PM
To: Chanaba, Nyree
Subject: revised htl itin


                                          AGENT TR/TR BOOKING REF ZNUHAJ
 
                                          ENRON NORTH AMERICA
                                          CAUSHOLLI/MONIKA
                                          3WTC0306
  ENRON CORP                              INTEROFFICE ETKT RCPT
 
 
 
 
 
  DATE:  OCT 03 2001
 
 
 
HOTEL                 21OCT CORONADO BAY RESORT
                      23OCT 4000 CORONADO BAY ROAD
                            CORONADO, CA 92118
                            UNITED STATES OF AMERICA
                            TELEPHONE: 619 424 4000
                            FAX: 619 424 4400
                            CONFIRMATION: 95683347
                            REFERENCE: A0DAD2
                            SINGLE ROOM KING SIZE BED
              

In [39]:
print(scenario_1["Message-Contents"][90])
for j in range(len(feature_names)):
        if scenario_1_tfidf[30][j] != 0:
            print(feature_names[j])
            print(scenario_1_tfidf[30][j])

Monika,
 
Please find tomorrow's Market Wrap attached.  You'll note that I modified the format slightly, but in general it's the same.  Have a great weekend and enjoy your time away from the office.  See you next week sometime.  Thanks.
 
KC
attached
0.125697373407
away
0.229114335521
enjoy
0.254559533455
format
0.246706584727
general
0.204580805782
great
0.182492396589
kc
0.350548034296
ll
0.161050993609
market
0.168364510163
modified
0.282887284158
monika
0.305039448285
note
0.174887968223
office
0.169018649246
slightly
0.294930065751
thanks
0.102804335074
time
0.125979905315
tomorrow
0.184805803857
week
0.152005552858
weekend
0.203703814195
wrap
0.330624327616


(116399, 12135)
30
35
38


KeyboardInterrupt: 

In [6]:
scenario_1_loaded = np.load('scenario_1_tfidf.npy')
print(scenario_1_loaded.shape)

KeyboardInterrupt: 

In [59]:
print(scenario_1_loaded)

NameError: name 'scenario_1_loaded' is not defined

In [42]:
scenario_2_tfidf_matrix = build_TFIDF_Matrix(scenario_2)
print(scenario_2_tfidf_matrix[1].toarray().shape)

scenario_2_tfidf = scenario_2_tfidf_matrix[1].toarray()
np.save('scenario_2_tfidf.npy', scenario_2_tfidf)
# scenario_2_data_frame = pd.DataFrame(data = scenario_2_tfidf, index = scenario_1["ID"], columns = scenario_2_tfidf_matrix[0].get_feature_names())

(116399, 12135)


In [43]:
scenario_3_tfidf_matrix = build_TFIDF_Matrix(scenario_3)
print(scenario_3_tfidf_matrix[1].toarray().shape)

scenario_3_tfidf = scenario_3_tfidf_matrix[1].toarray()
np.save('scenario_3_tfidf.npy', scenario_3_tfidf)
# scenario_3_data_frame = pd.DataFrame(data = scenario_3_tfidf, index = scenario_1["ID"], columns = scenario_3_tfidf_matrix[0].get_feature_names())

(116399, 12135)


AttributeError: get_feature_names not found

In [11]:
print(type(vectorizer))
print(type(vectorized))

NameError: name 'vectorizer' is not defined

## Testing code, small sample set

In [17]:
emails_short = emails[:20000]
vectorized_short = vectorizer.fit_transform(emails_short["Message-Contents"])
vectorized_short = pd.DataFrame(vectorized_short.toarray())
vectorized_short_T = vectorized_short.T
print(vectorized_short.shape)
feature_names = vectorizer.get_feature_names()
# print(len(feature_names))


(20000, 18104)


In [ ]:
feature_names.to_pickle('pickled_names.pickle')

In [11]:
print(type(vectorized_short))
print(vectorizer.get_feature_names()[:1000])

<class 'scipy.sparse.csr.csr_matrix'>
['000033', '000mt', '000s', '003366', '01730', '02081', '025866', '0337', '0354', '04075', '05500', '0551', '0748', '0f91f2936ed2981d3c03c3ee', '100080', '10013', '1004052048', '100mil', '10_year', '110101', '110180', '11112001', '112801_01_01', '112801_01_02', '112801_04_01', '112801_131_01', '112801_131_02', '112801_131_03', '113001', '11579', '116832', '121301', '1299', '13191369', '13214116', '13214487', '1365', '13897', '13905', '13906', '13914', '13917', '13920', '13924', '13938', '13943', '13947', '13950', '13955', '13959', '13966', '13968', '13969', '13972', '13988', '14364', '14367', '14370', '14382', '14383', '14388', '14389', '14390', '14392', '14393', '14399', '14400', '14401', '14404', '14410', '14414', '14420', '14421', '14471', '14472', '14481', '14483', '14485', '14492', '14494', '14495', '14503', '14515', '14519', '14525', '14526', '14542', '14543', '14544', '14566', '14567', '14568', '14570', '14573', '14582', '14586', '14587', '1

## LSA
We take the tf-idf matrix and run TruncatedSVD on it with whatever number of dimensions we want to reduce it to. The issue is I don't believe I can pull the categories. I can pull the highest ranking words from each category, but there is no storage of the category iteself. Currently a WIP on pulling each category, but for right now I'm doing the dot product so I can map term to term.

In [9]:
vectorizer = TfidfVectorizer(stop_words = stop_Words, min_df = .0005)

lsa = np.load('lsa_output_Jan25.npy')

In [7]:

# # Implemented without stemming to better work with current tf-idf code and the nasty documents

svd = TruncatedSVD(n_components = 100, algorithm='randomized')
# lsa = svd.fit_transform(scenario_1_tfidf)

def get_closest(term, vectorizer ,model):
 
    index = vectorizer.vocabulary_[term]

    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return vectorizer.get_feature_names()[out]
    else:
        return vectorizer.get_feature_names()[(out+1)]
    
def get_closest_document(index, vectorizer, model):
    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return emails_short["Message-Contents"][out]
    else:
        return emails_short["Message-Contents"][out+1]
#     if out<index:
#         return vectorizer.get_feature_names()[out]
#     else:
#         return vectorizer.get_feature_names()[(out+1)]

def get_best_document(index):
    pass
    
def get_k_closest(k,term,vectorizer,model):
    index = vectorizer.vocabulary_[term]
    print("index: ", index)

    # This is one approach where we take the dot product
    # This results in a mapping of terms to terms based on similarity to each other
    model = np.dot(model,model.T)
    print(model.shape)

    closest_terms = {}
    print(len(model))
    for i in range(len(model)):
        closest_terms[vectorizer.get_feature_names()[i]] = model[index][i]
    #print(closest_terms)
    sorted_list = sorted(closest_terms , key = lambda l : closest_terms[l])
    return sorted_list[::-1][0:k]
            

    


In [ ]:
# print(get_closest("ll", vectorizer, lsa))
# print(get_k_closest(5, "ll", vectorizer, lsa))

print(get_closest_document(70, vectorizer, lsa))

## get_closest_document example NOT MAINTAINED

Outputs the email that is the most similar to the one given (by index).

In [ ]:
print(get_closest_document(510, vectorizer, lsa))
print("--------------------------------------------------------------")
print(emails_short["Message-Contents"][510])

## Pulling the categories

Here we have the words that form the category for category \#50. "Monika" is the defining person of category \#50, as are "wrap", "market", and "report".

You can take absolute values 

In [27]:
# Load up the LSA output
svd_components = np.load('svd-components.npy')
print(type(svd_components))

'''
Return terms and values of the whole category with values rounded to the 
7th decimal place.
'''
def full_category(svd_components, feature_names, index):
    temp = list(reversed(svd_components[index].argsort()))
    svd_sorted = svd_components[index][temp]   
    print('\n'.join(["%0.7f*%s" % (coef, feat) for coef, feat in zip(svd_sorted, np.array(feature_names)[temp])]))

'''
Take the top 20 terms for each topic and save in a pandas dataframe. Store
the values in a separate dataframe
'''
def top_terms(svd_components, feature_names):
    term_columns = []
    value_columns = []
    for i in range(0, 20):
        term_columns.append("term" + str(i))
        value_columns.append("value" + str(i))
    terms_df = pd.DataFrame(columns=term_columns)
    values_df = pd.DataFrame(columns=value_columns)
    for i in range(0, 100):
        temp = list(reversed(svd_components[i].argsort()))
        svd_sorted = svd_components[i][temp]
        terms_row = []
        values_row = []
        for j in range(0, 20):
            terms_row.append(np.array(feature_names)[temp][j])
            values_row.append(svd_sorted[j])
        terms_df.loc[i] = terms_row
        values_df.loc[i] = values_row
    return terms_df, values_df         


full_category(svd_components, feature_names, 82)

# terms_df, values_df = top_terms(svd_components, feature_names)
# terms_df.to_pickle('LSA_dataframes/pickled_LSA_terms.pickle')
# values_df.to_pickle('LSA_dataframes/pickled_LSA_values.pickle')

<class 'numpy.ndarray'>


NameError: name 'feature_names' is not defined

In [26]:
sumTopic = 0
maxi = 0
topic = 10
doc = 6000
for i in range(0, 116399):
    maxi, new_i = max(lsa[i][topic], maxi), i
    

for i in range(0, 100):
    sumDoc += lsa[doc][i]
print(sumTopic, maxi)
print(new_i)
print(df["Message-Contents"][116398])

print(lsa.shape)

0 0.911805120124
116398
I forwarded this to Joe Parks - we both work for Ed McMichael.  I believe this is something he is monitoring.  Let me know if you have more questions.

 -----Original Message-----
From: 	Eubanks Jr., David W.  
Sent:	Wednesday, March 06, 2002 8:28 AM
To:	Germany, Chris; Dhont, Margaret
Subject:	Sitara deal 384247 - Apache Marketing

Chris,

I show a buy from Apache Marketing for February 2002, production on sitara deal 384247. Is this correct. Please advise.

Thanks

David W. Eubanks Jr
Specialist - Physical Gas Settlements
Enron Net Works, LLC
Phone  713-853-6678
Fax      713-646-8420
email: david.w.eubanks@enron.com

 << OLE Object: Picture (Device Independent Bitmap) >>
(116399, 100)


# Notes on known LSA bugs

Topic 03: Only finds log messages, which could be useless? Subjective, we should check whether any log messages are marked as relevant in the gold standard set.
Topic 19: finds mostly web addresses and weights them too much. We are not entirely removing links in our regex. One of the most important terms is "www" another is "http".
Topic 70: Created a category largely based around [IMAGE] tags. Pulls a bunch of junk emails with no real content.
Topic 72: The yahoo and jpg topic. More junk that needs to be taken out
Topic 82: More jpg issues, 

In [31]:
import pickle

with open('feature_names_short.pickle', 'wb') as f:
    pickle.dump(feature_names_short, f)

Empty DataFrame
Columns: [term0, term1, term2, term3, term4, term5, term6, term7, term8, term9, term10, term11, term12, term13, term14, term15, term16, term17, term18, term19]
Index: []
